# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fdae8e0a1f0>
├── 'a' --> tensor([[ 1.2228, -0.1741,  0.1163],
│                   [-0.4770, -1.3838,  0.2095]])
└── 'x' --> <FastTreeValue 0x7fdae8e0a070>
    └── 'c' --> tensor([[ 1.3899, -0.3018, -1.2745, -0.3082],
                        [-1.5126, -3.3403,  0.7599, -0.7156],
                        [ 0.7037,  0.8844,  0.1097,  0.3929]])

In [4]:
t.a

tensor([[ 1.2228, -0.1741,  0.1163],
        [-0.4770, -1.3838,  0.2095]])

In [5]:
%timeit t.a

60.8 ns ± 0.0209 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fdae8e0a1f0>
├── 'a' --> tensor([[ 0.0228,  2.1305,  0.7207],
│                   [ 0.7568, -1.0735, -0.5591]])
└── 'x' --> <FastTreeValue 0x7fdae8e0a070>
    └── 'c' --> tensor([[ 1.3899, -0.3018, -1.2745, -0.3082],
                        [-1.5126, -3.3403,  0.7599, -0.7156],
                        [ 0.7037,  0.8844,  0.1097,  0.3929]])

In [7]:
%timeit t.a = new_value

57.5 ns ± 0.041 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2228, -0.1741,  0.1163],
               [-0.4770, -1.3838,  0.2095]]),
    x: Batch(
           c: tensor([[ 1.3899, -0.3018, -1.2745, -0.3082],
                      [-1.5126, -3.3403,  0.7599, -0.7156],
                      [ 0.7037,  0.8844,  0.1097,  0.3929]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2228, -0.1741,  0.1163],
        [-0.4770, -1.3838,  0.2095]])

In [11]:
%timeit b.a

52.6 ns ± 0.0273 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.6453, -0.7026,  0.4988],
               [-0.3996, -0.4150, -0.1109]]),
    x: Batch(
           c: tensor([[ 1.3899, -0.3018, -1.2745, -0.3082],
                      [-1.5126, -3.3403,  0.7599, -0.7156],
                      [ 0.7037,  0.8844,  0.1097,  0.3929]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.0805 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

847 ns ± 0.165 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 28 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 1.53 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 470 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fdaebe65c10>
├── 'a' --> tensor([[[ 1.2228, -0.1741,  0.1163],
│                    [-0.4770, -1.3838,  0.2095]],
│           
│                   [[ 1.2228, -0.1741,  0.1163],
│                    [-0.4770, -1.3838,  0.2095]],
│           
│                   [[ 1.2228, -0.1741,  0.1163],
│                    [-0.4770, -1.3838,  0.2095]],
│           
│                   [[ 1.2228, -0.1741,  0.1163],
│                    [-0.4770, -1.3838,  0.2095]],
│           
│                   [[ 1.2228, -0.1741,  0.1163],
│                    [-0.4770, -1.3838,  0.2095]],
│           
│                   [[ 1.2228, -0.1741,  0.1163],
│                    [-0.4770, -1.3838,  0.2095]],
│           
│                   [[ 1.2228, -0.1741,  0.1163],
│                    [-0.4770, -1.3838,  0.2095]],
│           
│                   [[ 1.2228, -0.1741,  0.1163],
│                    [-0.4770, -1.3838,  0.2095]]])
└── 'x' --> <FastTreeValue 0x7fda327c7220>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.2 µs ± 77 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fda38d16e80>
├── 'a' --> tensor([[ 1.2228, -0.1741,  0.1163],
│                   [-0.4770, -1.3838,  0.2095],
│                   [ 1.2228, -0.1741,  0.1163],
│                   [-0.4770, -1.3838,  0.2095],
│                   [ 1.2228, -0.1741,  0.1163],
│                   [-0.4770, -1.3838,  0.2095],
│                   [ 1.2228, -0.1741,  0.1163],
│                   [-0.4770, -1.3838,  0.2095],
│                   [ 1.2228, -0.1741,  0.1163],
│                   [-0.4770, -1.3838,  0.2095],
│                   [ 1.2228, -0.1741,  0.1163],
│                   [-0.4770, -1.3838,  0.2095],
│                   [ 1.2228, -0.1741,  0.1163],
│                   [-0.4770, -1.3838,  0.2095],
│                   [ 1.2228, -0.1741,  0.1163],
│                   [-0.4770, -1.3838,  0.2095]])
└── 'x' --> <FastTreeValue 0x7fdae8e39e20>
    └── 'c' --> tensor([[ 1.3899, -0.3018, -1.2745, -0.3082],
                        [-1.5126, -3.3403,  0.7599, -0.7156],
                 

In [23]:
%timeit t_cat(trees)

31.1 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.1 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.2228, -0.1741,  0.1163],
                [-0.4770, -1.3838,  0.2095]],
       
               [[ 1.2228, -0.1741,  0.1163],
                [-0.4770, -1.3838,  0.2095]],
       
               [[ 1.2228, -0.1741,  0.1163],
                [-0.4770, -1.3838,  0.2095]],
       
               [[ 1.2228, -0.1741,  0.1163],
                [-0.4770, -1.3838,  0.2095]],
       
               [[ 1.2228, -0.1741,  0.1163],
                [-0.4770, -1.3838,  0.2095]],
       
               [[ 1.2228, -0.1741,  0.1163],
                [-0.4770, -1.3838,  0.2095]],
       
               [[ 1.2228, -0.1741,  0.1163],
                [-0.4770, -1.3838,  0.2095]],
       
               [[ 1.2228, -0.1741,  0.1163],
                [-0.4770, -1.3838,  0.2095]]]),
    x: Batch(
           c: tensor([[[ 1.3899, -0.3018, -1.2745, -0.3082],
                       [-1.5126, -3.3403,  0.7599, -0.7156],
                       [ 0.7037,  0.8844,  0.1097,  0.3929]],
         

In [26]:
%timeit Batch.stack(batches)

77.5 µs ± 283 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.2228, -0.1741,  0.1163],
               [-0.4770, -1.3838,  0.2095],
               [ 1.2228, -0.1741,  0.1163],
               [-0.4770, -1.3838,  0.2095],
               [ 1.2228, -0.1741,  0.1163],
               [-0.4770, -1.3838,  0.2095],
               [ 1.2228, -0.1741,  0.1163],
               [-0.4770, -1.3838,  0.2095],
               [ 1.2228, -0.1741,  0.1163],
               [-0.4770, -1.3838,  0.2095],
               [ 1.2228, -0.1741,  0.1163],
               [-0.4770, -1.3838,  0.2095],
               [ 1.2228, -0.1741,  0.1163],
               [-0.4770, -1.3838,  0.2095],
               [ 1.2228, -0.1741,  0.1163],
               [-0.4770, -1.3838,  0.2095]]),
    x: Batch(
           c: tensor([[ 1.3899, -0.3018, -1.2745, -0.3082],
                      [-1.5126, -3.3403,  0.7599, -0.7156],
                      [ 0.7037,  0.8844,  0.1097,  0.3929],
                      [ 1.3899, -0.3018, -1.2745, -0.3082],
                      [-1.5126, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 412 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 3.46 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
